# <b>背景</b>

<p>ET（essential tremor）患者：原发性震颤(Essential tremor)患者，特发性震颤（essential tremor，ＥＴ）最常见的运动障碍性疾病，主要为手、头部及身体其他部位的姿位性和运动性震颤。发病部位：上肢、头、面部、下颚。</p>
<p>参见文献： louis2003 Factors associated with increased risk of head tremor in essential tremor_ a community-based study in northern Manhattan）。</p>
<p>中线震颤（midline tremor）：   包含：面部（下颌部+唇部）、舌头、声音、头部（又称颈部）和躯干。<p>

    
# <b>目的</b>
1.1.	探索ET患者伴中线震颤的危险因素。
    
1.2.	ET患者伴焦虑和抑郁的危险因素    
    
# <b>任务</b>

这份数据来自内科，手工输入导致数据缺失严重或中英文混乱等问题，数据清理工作较为繁琐。

# <b>第一步 引包</b>

In [ ]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve,auc
import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import time
import warnings
warnings.filterwarnings('ignore')

# <b>第二步 读取excel数据</b>

1）设置默认目录

2）通过表单名查找数据

3）读取该表单中所有数据

In [ ]:
dir = "./"
print(os.listdir(dir))

In [ ]:
# 读取excel数据
df = pd.DataFrame()
file = "data/ET虚构数据.xlsx"
data = xlrd.open_workbook(dir+file).sheet_by_name("总表")
df = pd.read_excel(dir+file,data.name,index_col=None,na_values=['9999'])
df.shape

# <b>第三步 探索数据 </b>
<p>了解数据分布和数据质量</p>
<b>1) 查看字段名</b>

In [ ]:
df.columns   #查看字段名

<p><b>发现：</b></p>
<p>---字段名有重复; </p>
<p>---字段名中带了中英文符号和注解，不方便后面使用列名。 </p>
<p><b>2) 查看数据类型 </b></p>

In [ ]:
#-------DataType of columns
#print(df.dtypes)  #  [25:55]       #view  the columns types           
dtype_df = df.dtypes.reset_index()    # calculate the types stat 
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index() 

In [ ]:
df.info()

<b>3) 查看值域</b>

In [ ]:
df1 = df.iloc[:,2:]
for i in df1.columns:
    uniquevalue = str(df1[i].unique())
    print (i,":  ",uniquevalue)

<p><b> 发现 </b></p>
--病程字段：有“xx月”，空值，“查体见”，'自述，但起病时间不详'等取值，影响字段转成数值型继续进行分析；
--

<b>4）数据缺失情况 </b>

In [ ]:
#missing data
print ("-----------------------------总记录数： " ,df.index.size,"条    --------------------------")
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['missing_count', 'missing_Percent'])
missing_data.head(20)

<p><b>发现：</b></p>
<p>---xx病程缺失较多：缺失值为未知病程或未出现该震颤; </p>
<p>---xx震颤：缺失值为未出现该震颤。 </p>

# <b>第四步 预处理 </b>
数据清理/预处理是一个非常繁琐的工作，根据业务需求和数据质量而不同，没有标准的流程。下面根据实列来了解常用的一些操作：

<b>1）删除重复字段</b>

从“检查日期”中提取年份信息

In [ ]:
x = [2]  # 两个 性别 字段 重复了
cols = df.columns[x]
print (" 重复列,共计",len(cols),"个字段:    " , cols)
df.drop(cols,axis=1,inplace=True)

<b> 2）删除字段名称中无用字符 </b>

In [ ]:
df.columns = [re.sub('（.*?）', '', hdr)  for hdr in df.columns]  #删除 中文括号及括号内字符
df.columns = [re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]", '', hdr)  for hdr in df.columns]  #删除 英文(半角)符号及符号内字符
df.columns = [re.sub("）", '', hdr)  for hdr in df.columns]   #删除 未匹配的中文括号
print("data rows and columns : ", df.shape )    #209条数据，57个字段

In [ ]:
df.columns

<b> 3) 病程字段清洗 </b>


In [ ]:
# 替换数值型缺失值为零
df.loc[:,["总病程", "上肢病程", "下肢病程","颈部病程", "声音病程", "面部病程", "舌病程", "躯干病程"]].fillna(0)
df.head(2)

In [ ]:
#替换字符串缺失值为零
for i in ["总病程", "上肢病程", "下肢病程","颈部病程", "声音病程", "面部病程", "舌病程", "躯干病程"]:
    df.loc[df[i].astype(str)=="",i] =0
    df.loc[df[i].astype(str) =="nan",i] = 0
    df.loc[df[i].astype(str) == "查体见",i] = ""   #临时补充的，后期可能有其他方式处理
df.head(2)

In [ ]:
for i in ["总病程", "上肢病程", "下肢病程","颈部病程", "声音病程", "面部病程", "舌病程", "躯干病程"]:
    
    df[i] = np.where(df[i].astype(str).str.contains(r"\d"),df[i],"")   #不含数值的字符串替换为空值
    df.loc[df[i].astype(str)=="",i] =0.001                             #将空值设置为任意值，例如0.001，避免字段转成数值型时，缺失值导致报错
    df[i] = np.where(df[i].astype(str).str.contains("月"),df[i].astype(str).apply(lambda x:str(float(re.sub("\D","",x))/12)),df[i])  #含“月”的取值，只取数值
    df[i] = df[i].astype(float)                                      # 将只含数值的列转成浮点数值型
df.head(2)

# <b>第五步 导出结果 </b>


In [ ]:
df.to_csv(dir+"output/df_clean.csv")